<a href="https://colab.research.google.com/github/fabiodr/colabs/blob/main/optillm_decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# Install the optillm library from github
!pip install --no-cache-dir -U git+https://github.com/codelion/optillm.git

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optillm.cot_decoding import cot_decode
from optillm.entropy_decoding import entropy_decode
import torch

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

messages = [
    {"role": "user", "content": "In a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enrolled in jazz dance, and the rest enrolled in hip-hop dance. What percentage of the entire students enrolled in hip-hop dance?"}
]

# Generate the response using CoT decoding
result, confidence = cot_decode(model, tokenizer, messages, aggregate_paths=True, max_new_tokens=512)
print(f"CoT Decoding:\n {result}")
# print(f"Confidence: {confidence}")

# Generate the response using Entropy decoding
result = entropy_decode(model, tokenizer, messages)
print(f"Entropy Decoding:\n {result}")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
Qwen2Model is using Qwen2SdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


CoT Decoding:
 To determine the percentage of the entire dance class enrolled in hip-hop dance, we will follow these steps:

1. **Calculate the number of students enrolled in contemporary dance:**
   20% of 20 students enrolled in contemporary dance.
   \[
   0.20 \times 20 = 4 \text{ students}
   \]

2. **Calculate the number of remaining students after those enrolled in contemporary dance:**
   Subtract the number of students enrolled in contemporary dance from the total number of students.
   \[
   20 - 4 = 16 \text{ students}
   \]

3. **Calculate the number of students enrolled in jazz dance:**
   25% of 16 students enrolled in hip-hop dance.
   \[
   0.25 \times 16 = 4 \text{ students}
   \]

4. **Calculate the number of remaining students after those enrolled in jazz dance:**
   Subtract the number of students enrolled in jazz dance from the remainder after those enrolled in contemporary dance.
   \[
   16 - 4 = 12 \text{ students}
   \]

5. **Calculate the percentage of the ent

/usr/local/lib/python3.10/dist-packages/optillm/entropy_decoding.py:32: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1808.)
  attn_varentropy = torch.var(attn_entropy, dim=-1)


Entropy Decoding:
 To determine the percentage of the entire dance class enrolled in hip-hop dance, we need to follow these steps:

1. Calculate the number of students enrolled in contemporary dance.
2. Determine the number of students remaining after removing those enrolled in contemporary dance.
3. Calculate the number of students enrolled in jazz dance from the remaining students.
4. Determine the number of students remaining after removing those enrolled in jazz dance.
5. Finally, calculate the percentage of the entire dance class enrolled in hip-hop dance.

Let's start with step 1:
The number of students enrolled in contemporary dance is 20% of 20 students.
\[ 0.20 \times 20 = 4 \]
So, 4 students are enrolled in contemporary dance.

Next, we subtract the number of students enrolled in contemporary dance from the total number of students to find the number of students remaining after adding those who are enrolled in jazz dance.
\[ 20 - 4 = 16 \]
So, 16 students are enrolled in jazz

In [ ]:
# Greedy decoding

def get_device():
  if torch.cuda.is_available():
    return torch.device("cuda")
  else:
    return torch.device("cpu")

device = get_device()
model = model.to(device)
# print(f"Using device: {device}")
input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
input_ids = input_ids.to(device)
input_length = input_ids.shape[1]
output_ids = model.generate(
    input_ids,
    max_new_tokens=512,  # Adjust this value based on how long you want the response to be
    do_sample=False,     # This ensures greedy decoding
    num_beams=1,
)
output_ids = output_ids.cpu()
# Decode only the newly generated tokens
response = tokenizer.decode(output_ids[0][input_length:], skip_special_tokens=True)
print(f"Greedy Decoding:\n {response}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:589: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos toke

Greedy Decoding:
 To determine the percentage of the entire students enrolled in hip-hop dance, we need to follow these steps:

1. Calculate the number of students enrolled in contemporary dance.
2. Determine how many students remain after subtracting those who are enrolled in contemporary dance.
3. Calculate the number of students enrolled in jazz dance from the remaining students.
4. Find out how many students remain after subtracting those who are enrolled in jazz dance.
5. Determine the number of students enrolled in hip-hop dance from the remaining students.
6. Convert this number into a percentage of the total number of students.

Let's start with step 1:
- The total number of students is 20.
- 20% of the students enroll in contemporary dance.
\[ \text{Number of students in contemporary dance} = 0.20 \times 20 = 4 \]

Next, step 2:
- After subtracting the 4 students in contemporary dance, the number of students left is:
\[ 20 - 4 = 16 \]

Now, step 3:
- 25% of the remaining 16 st